In [ ]:
# Install required libraries silently
!pip install -q \
    langchain \
    langchain-community \
    langchain-huggingface \
    transformers \
    sentence-transformers \
    faiss-cpu \
    accelerate \
    bitsandbytes \
    requests==2.32.4 \
    huggingface-hub \
    google-search-results # <--- THE FIX IS FORCING THE VERSION ABOVE

import torch
import pandas as pd
import numpy as np
import os
import requests
from google.colab import userdata
from huggingface_hub import login

# LangChain Imports
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace
from langchain_community.utilities import GoogleSerperAPIWrapper

# Transformers Imports (for 4-bit loading)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

# ----------------------------------------------------------------------
# 1. Secure API Key Setup
# ----------------------------------------------------------------------
print("Attempting to set API keys...")

# --- Hugging Face Login & Token ---
hf_token = userdata.get('HF_TOKEN')
if hf_token:
    login(token=hf_token, add_to_git_credential=False)
    os.environ['HF_TOKEN'] = hf_token
    print("✅ Successfully logged in to Hugging Face and set HF_TOKEN env variable.")
else:
    print("❌ HF_TOKEN secret not found. Model loading in Cell 3 will fail.")

# --- Serper (Web Search) API Key ---
serper_api_key = userdata.get('SERPER_API_KEY')
if serper_api_key:
    os.environ['SERPER_API_KEY'] = serper_api_key
    print("✅ Successfully set SERPER_API_KEY env variable.")
else:
    print("⚠️ WARNING: SERPER_API_KEY secret not found. Web search will be skipped.")
    print("Please set 'SERPER_API_KEY' in Colab Secrets for web-augmented results.")


print("\n--- Dependencies Installed and Keys Configured ---")

In [ ]:
# ----------------------------------------------------------------------
# 2. Data, Embeddings, and Vector Store Setup
# ----------------------------------------------------------------------

# Small sample of role descriptions
data = {
    "role": ["Data Analyst", "Software Engineer", "Business Analyst", "AI Engineer", "UI/UX Designer"],
    "description": [
        "Analyze datasets to derive insights, build dashboards, and support decision making.",
        "Develop and maintain software systems, write efficient code, test and debug.",
        "Bridge business needs and data; communicate insights to stakeholders.",
        "Design and train machine learning models, optimize algorithms.",
        "Design user interfaces and experiences that are intuitive and appealing."
    ]
}
df = pd.DataFrame(data)

# 1. Convert DataFrame to LangChain Document objects
# This stores the 'role' as metadata for potential future use
documents = [
    Document(
        page_content=row["description"],
        metadata={"role": row["role"]}
    ) for _, row in df.iterrows()
]
print(f"Created {len(documents)} LangChain Documents.")

# 2. Load Embedding Model
embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)

# 3. Create FAISS Vector Store
# This handles embedding the documents and indexing them in one step
try:
    vector_store = FAISS.from_documents(documents, embeddings)
    print("\nFAISS vector store created successfully.")

    # 4. Create Retriever
    # This object is the "search function" for our RAG chain
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    print("Retriever object created.")

    # 5. Initialize Web Search Tool
    search_tool = GoogleSerperAPIWrapper()
    # You can test it with: print(search_tool.run("what is langchain?"))
    print("Google Serper web search tool initialized.")

except Exception as e:
    print(f"\n❌ Error creating vector store: {e}")
    print("This might be a network issue or an problem with the embedding model.")

print("\n--- Knowledge Base and Tools Ready ---")

In [ ]:
# ----------------------------------------------------------------------
# 3. LLM Setup (Mistral-7B-Instruct-v0.3)
# ----------------------------------------------------------------------

# --- FIX: Moved the new import to the top of the cell ---
from langchain_huggingface import HuggingFacePipeline

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

try:
    # 4-bit Quantization configuration
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load model using 4-bit quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.float16,
        trust_remote_code=True
    )

    # Initialize the transformers pipeline
    gen_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        temperature=0.7,
        do_sample=True,
        return_full_text=False  # <--- THIS IS THE FIX
    )

    # --- LangChain Wrapper (FIXED INDENTATION) ---
    # This logic is now correctly indented INSIDE the try block

    # 1. Wrap the raw transformers pipeline in LangChain's HuggingFacePipeline
    llm_pipeline = HuggingFacePipeline(pipeline=gen_pipeline)

    # 2. Wrap the LangChain pipeline in ChatHuggingFace for chat message compatibility
    chat_llm = ChatHuggingFace(llm=llm_pipeline)
    # ---------------------------------

    print(f"\n✅ Model {model_name} loaded and wrapped for LangChain successfully (4-bit).")

except Exception as e:
    print(f"\n❌ ERROR: Model loading failed. ")
    print(f"Error details: {e}")
    chat_llm = None

In [ ]:
# ----------------------------------------------------------------------
# 4. LangChain RAG Chain (Replaces pathfinder_agent)
# ----------------------------------------------------------------------

if chat_llm:
    # 1. Define helper functions
    def format_docs(docs):
        """Converts a list of Document objects into a single string."""
        return "\n".join(
            f"{doc.metadata.get('role', 'Source')}: {doc.page_content}" for doc in docs
        )

    # 2. Define the context retrieval step
    context_retriever = RunnableParallel(
        related_roles=retriever | format_docs,
        web_info=RunnableLambda(lambda x: search_tool.run(x + " career path salary india")),
        profile_text=RunnablePassthrough() # Passes the original profile text through
    )

    # 3. Define the Prompt (--- THIS IS THE UPGRADED PROMPT ---)
    # The template uses the keys from context_retriever ('related_roles', 'web_info', 'profile_text')
    system_prompt = """
You are Pathfinder AI, an expert career counsellor.
Your task is to analyze the user's profile, especially their **likes, dislikes, and constraints**, and provide a structured career plan.

Use the context below, but **you must critically evaluate if the retrieved roles fit the user's constraints.**
If a retrieved role (like "AI Engineer") is a **bad fit** due to the user's constraints (e.g., "dislikes math"), you **must recommend against it** or suggest an alternative, less-known role that fits better (like "MLOps Engineer" or "AI Product Manager").

Relevant job info:
{related_roles}

Recent web info:
{web_info}

Follow the steps exactly:
1. **Critically analyze the user's profile.**
2. Identify 2-3 **genuinely suitable** career roles. **Prioritize the user's constraints (likes/dislikes) over their general interests.**
3. For each role, explain *why* it is a good fit, specifically addressing their constraints.
4. List 3 key missing skills for each role.
5. Suggest a 6-month learning roadmap with *specific, appropriate* courses and projects that respect the user's constraints (e.g., recommend "AI for Everyone" not a math-heavy course).

Format the final output clearly using headings and bullet points.
"""

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "My profile is: {profile_text}")
    ])

    # 4. Build the Full RAG Chain
    # This is the complete, runnable agent
    rag_chain = (
        context_retriever |   # 1. Get context
        prompt_template   |   # 2. Format prompt
        chat_llm          |   # 3. Call LLM
        StrOutputParser()     # 4. Get string output
    )

    print("✅ LangChain RAG chain (LCEL) built successfully with **upgraded prompt**.")

else:
    print("❌ LLM not loaded. Cannot create RAG chain.")
    rag_chain = None

In [ ]:
# ----------------------------------------------------------------------
# 5. Execution
# ----------------------------------------------------------------------

profile = """I am a 3rd year B.Tech CSE student.
I enjoy working with data, love research and visualization,
but dislike hardcore coding. I prefer hybrid or remote jobs."""

print(f"--- Processing Profile ---")
print(profile)

if rag_chain:
    # .invoke() runs the entire chain
    result = rag_chain.invoke(profile)

    print("\n--- Pathfinder AI Result (via LangChain) ---")
    print(result)
else:
    print("\n--- Skipping execution as chain failed to build ---")

In [ ]:
%%writefile app.py
# -------------------------------------------------------------------
# LangChain Version - RAG Agent Streamlit App
# -------------------------------------------------------------------
import streamlit as st
import pandas as pd
import numpy as np
import faiss
import torch
import os
import time

# Hugging Face & Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from huggingface_hub import login

# LangChain Imports
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFacePipeline
from langchain_community.utilities import GoogleSerperAPIWrapper

# --- Configuration ---
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"
EMBEDDING_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'

# -------------------------------------------------------------------
# 1. Resource Loading (@st.cache_resource)
# -------------------------------------------------------------------

@st.cache_resource
def load_llm():
    """Loads the 4-bit quantized LLM and wraps it for LangChain."""

    # Read token from OS environment (set by the launch script)
    hf_token = os.environ.get('HF_TOKEN')
    if hf_token:
        login(token=hf_token, add_to_git_credential=False)
    else:
        st.error("HF_TOKEN environment variable not set. Cannot load model.")
        return None

    try:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
        gen_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=1024,
            temperature=0.7,
            do_sample=True,
            return_full_text=False  # <--- THIS IS THE FIX
        )

        # --- Wrap for LangChain (FIXED) ---
        llm_pipeline = HuggingFacePipeline(pipeline=gen_pipeline)
        chat_llm = ChatHuggingFace(llm=llm_pipeline)
        return chat_llm

    except Exception as e:
        st.error("Failed to load LLM. Error: {}".format(e))
        return None

@st.cache_resource
def load_embedding_model():
    """Loads the embedding model."""
    try:
        return HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
    except Exception as e:
        st.error("Failed to load embedding model. Error: {}".format(e))
        return None

@st.cache_resource
def create_knowledge_base(_embed_model): # <-- FIX: Added underscore to ignore for hashing
    """Creates the DataFrame and the FAISS vector store."""
    data = {
        "role": ["Data Analyst", "Software Engineer", "Business Analyst", "AI Engineer", "UI/UX Designer"],
        "description": [
            "Analyze datasets to derive insights, build dashboards, and support decision making.",
            "Develop and maintain software systems, write efficient code, test and debug.",
            "Bridge business needs and data; communicate insights to stakeholders.",
            "Design and train machine learning models, optimize algorithms.",
            "Design user interfaces and experiences that are intuitive and appealing."
        ]
    }

    documents = [
        Document(page_content=desc, metadata={"role": role})
        for role, desc in zip(data["role"], data["description"])
    ]

    vector_store = FAISS.from_documents(documents, _embed_model)
    return vector_store

# -------------------------------------------------------------------
# 2. RAG Chain Definition
# -------------------------------------------------------------------

def build_rag_chain(chat_llm, retriever, search_tool):
    """Builds the executable LangChain RAG agent chain."""

    def format_docs(docs):
        return "\n".join(
            f"{doc.metadata.get('role', 'Source')}: {doc.page_content}" for doc in docs
        )

    context_retriever = RunnableParallel(
        related_roles=retriever | format_docs,
        web_info=RunnableLambda(lambda x: search_tool.run(x + " career path salary india")),
        profile_text=RunnablePassthrough()
    )

    # --- THIS IS THE UPGRADED PROMPT ---
    system_prompt = """
You are Pathfinder AI, an expert career counsellor.
Your task is to analyze the user's profile, especially their **likes, dislikes, and constraints**, and provide a structured career plan.

Use the context below, but **you must critically evaluate if the retrieved roles fit the user's constraints.**
If a retrieved role (like "AI Engineer") is a **bad fit** due to the user's constraints (e.g., "dislikes math"), you **must recommend against it** or suggest an alternative, less-known role that fits better (like "MLOps Engineer" or "AI Product Manager").

Relevant job info:
{related_roles}

Recent web info:
{web_info}

Follow the steps exactly:
1. **Critically analyze the user's profile.**
2. Identify 2-3 **genuinely suitable** career roles. **Prioritize the user's constraints (likes/dislikes) over their general interests.**
3. For each role, explain *why* it is a good fit, specifically addressing their constraints.
4. List 3 key missing skills for each role.
5. Suggest a 6-month learning roadmap with *specific, appropriate* courses and projects that respect the user's constraints (e.g., recommend "AI for Everyone" not a math-heavy course).

Format the final output clearly using headings and bullet points.
"""

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "My profile is: {profile_text}")
    ])

    rag_chain = (
        context_retriever |
        prompt_template |
        chat_llm |
        StrOutputParser()
    )

    return rag_chain

# -------------------------------------------------------------------
# 3. Streamlit UI
# -------------------------------------------------------------------

def main():
    st.set_page_config(page_title="Pathfinder AI Career Counselor", layout="wide")
    st.title("🤖 Pathfinder AI Career Counselor")
    st.caption("LangChain RAG Agent powered by Mistral-7B-Instruct-v0.3")

    # --- Load all resources ---
    chat_llm = load_llm()
    embed_model = load_embedding_model()

    if not chat_llm or not embed_model:
        st.error("Core models failed to load. The application cannot start.")
        st.stop()

    # Check for Serper key
    if not os.environ.get('SERPER_API_KEY'):
        st.warning("SERPER_API_KEY not set. Web search results will be disabled.", icon="⚠️")
        search_tool = RunnableLambda(lambda x: "Web search disabled. SERPER_API_KEY not provided.")
    else:
        search_tool = GoogleSerperAPIWrapper()

    vector_store = create_knowledge_base(_embed_model=embed_model)
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})

    # --- Build the single, reusable RAG chain ---
    try:
        rag_chain = build_rag_chain(chat_llm, retriever, search_tool)
    except Exception as e:
        st.error("Failed to build RAG chain: {}".format(e))
        st.stop()


    # --- Chat Interface ---
    if "messages" not in st.session_state:
        st.session_state.messages = []

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    if profile_query := st.chat_input("Paste your career profile here to get an analysis..."):
        st.session_state.messages.append({"role": "user", "content": profile_query})

        with st.chat_message("user"):
            st.markdown(profile_query)

        with st.chat_message("assistant"):
            with st.spinner("Analyzing profile and generating career roadmap... This may take up to 30 seconds."):

                # --- Run the chain ---
                response = rag_chain.invoke(profile_query)

                st.markdown(response)

            st.session_state.messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
    main()

In [ ]:
# Install Ngrok and Streamlit libraries
!pip install -q streamlit pyngrok \
    langchain \
    langchain-community \
    langchain-huggingface \
    transformers \
    sentence-transformers \
    faiss-cpu \
    accelerate \
    bitsandbytes \
    requests==2.32.4 \
    huggingface-hub \
    google-search-results

# Authenticate Ngrok and set environment variables
from pyngrok import ngrok
from google.colab import userdata
import time
import os

# --- 1. SET ENVIRONMENT VARIABLES ---
HF_TOKEN = userdata.get('HF_TOKEN')
if HF_TOKEN:
    os.environ['HF_TOKEN'] = HF_TOKEN
    print("✅ HF_TOKEN loaded into environment variables.")
else:
    print("❌ HF_TOKEN secret not found. Check Colab Secrets.")

# NEW: Set Serper API Key
SERPER_API_KEY = userdata.get('SERPER_API_KEY')
if SERPER_API_KEY:
    os.environ['SERPER_API_KEY'] = SERPER_API_KEY
    print("✅ SERPER_API_KEY loaded into environment variables.")
else:
    print("⚠️ WARNING: SERPER_API_KEY secret not found. Web search in the app will be disabled.")


# --- 2. AUTHENTICATE NGROK ---
NGROK_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ Ngrok Authenticated.")
else:
    print("❌ NGROK_AUTH_TOKEN secret not found. Cannot start tunnel. Check Colab Secrets.")

# Disconnect any previous tunnels
try:
    ngrok.disconnect(ngrok.get_public_url())
    print("✅ Successfully disconnected previous Ngrok tunnel (if any).")
except:
    print("No active tunnels found, proceeding to launch.")

# --- 3. START STREAMLIT AND NGROK ---
print("Starting Streamlit server...")

# Kill any previous Streamlit process
!killall streamlit >/dev/null 2>&1
time.sleep(2)

# Launch Streamlit. We pass BOTH tokens to the environment.
!env HF_TOKEN=$HF_TOKEN SERPER_API_KEY=$SERPER_API_KEY nohup streamlit run app.py --server.port 8501 &

# Give the server a moment to start
time.sleep(10)

# Get the Ngrok public URL
try:
    public_url = ngrok.connect(addr='8501', proto='http')
    print("\n--- Streamlit App is Running! 🚀 ---")
    print("Click the link below to access your Pathfinder AI Web App:")
    print(public_url)
except Exception as e:
    print(f"\n❌ Error connecting Ngrok tunnel. The Streamlit server may have failed to start. Error: {e}")